## 1️⃣ Setup - Clonar Repositório e Verificar GPU

In [ ]:
# Clonar repositório do GitHub
!git clone https://github.com/gabrielamds/k-means-1d.git
%cd k-means-1d

# Verificar GPU disponível
!nvidia-smi --query-gpu=name,memory.total,compute_cap --format=csv

## 2️⃣ Gerar Datasets

Gera os mesmos dados usados no Windows (seeds 42, 43, 44):

In [ ]:
# Gerar os 3 datasets
%cd data
!python3 generate_data.py --N 10000 --K 4 --output dados_pequeno --seed 42
!python3 generate_data.py --N 100000 --K 8 --output dados_medio --seed 43
!python3 generate_data.py --N 1000000 --K 16 --output dados_grande --seed 44

# Confirmar que foram criados
print("\nDatasets criados:")
!ls -lh *.csv
%cd ..

## 3️⃣ Compilar CUDA

In [ ]:
%%bash
cd cuda
nvcc -O2 -arch=sm_75 kmeans_1d_cuda.cu -o kmeans_1d_cuda
echo "✓ CUDA compilado com sucesso"
ls -lh kmeans_1d_cuda

---

# 📊 PARTE 1: CUDA PURO

## Benchmark CUDA - 3 Datasets

In [ ]:
%%bash
echo "========================================="
echo "BENCHMARK: CUDA"
echo "========================================="

echo ""
echo "--- Dataset PEQUENO (10K, K=4) ---"
cuda/kmeans_1d_cuda data/dados_pequeno.csv data/dados_pequeno_centroides_init.csv 50 1e-6 256

echo ""
echo "--- Dataset MÉDIO (100K, K=8) ---"
cuda/kmeans_1d_cuda data/dados_medio.csv data/dados_medio_centroides_init.csv 50 1e-6 256

echo ""
echo "--- Dataset GRANDE (1M, K=16) ---"
cuda/kmeans_1d_cuda data/dados_grande.csv data/dados_grande_centroides_init.csv 50 1e-6 256

## Testar Diferentes Configurações (Threads per Block)

In [ ]:
%%bash
echo "=== Variando Threads per Block (Dataset Grande) ==="
for TPB in 128 256 512 1024; do
    echo ""
    echo "Threads/Block: $TPB"
    cuda/kmeans_1d_cuda data/dados_grande.csv data/dados_grande_centroides_init.csv 50 1e-6 $TPB | grep "Tempo"
done

---

# 📊 PARTE 2: HYBRID (OpenMP + CUDA)

## Compilar OpenMP + CUDA

In [ ]:
%%bash
cd hybrid
nvcc -O2 -arch=sm_75 -Xcompiler -fopenmp kmeans_1d_omp_cuda.cu -o kmeans_1d_omp_cuda
echo "✓ OpenMP + CUDA compilado"
ls -lh kmeans_1d_omp_cuda

## Benchmark OpenMP + CUDA

In [ ]:
%%bash
echo "========================================="
echo "BENCHMARK: HYBRID (OpenMP + CUDA)"
echo "========================================="

datasets=("pequeno:10K:4" "medio:100K:8" "grande:1M:16")

for ds in "${datasets[@]}"; do
    IFS=':' read -r name N K <<< "$ds"
    echo ""
    echo "=== Dataset: ${name^^} (N=$N, K=$K) ==="
    
    for THREADS in 1 2 4 8; do
        echo "  OpenMP $THREADS threads + CUDA:"
        OMP_NUM_THREADS=$THREADS hybrid/kmeans_1d_omp_cuda \
            data/dados_$name.csv data/dados_${name}_centroides_init.csv \
            50 1e-6 256 | grep "Tempo"
    done
done

---

# 📊 PARTE 4: HYBRID (MPI + CUDA)

In [ ]:
%%bash
echo "========================================="
echo "BENCHMARK: HYBRID (OpenMP + MPI)"
echo "========================================="

datasets=("pequeno:10K:4" "medio:100K:8" "grande:1M:16")

for ds in "${datasets[@]}"; do
    IFS=':' read -r name N K <<< "$ds"
    echo ""
    echo "=== Dataset: ${name^^} (N=$N, K=$K) ==="
    
    echo "  1 processo MPI x 2 threads OpenMP:"
    OMP_NUM_THREADS=2 mpirun -np 1 --allow-run-as-root --oversubscribe \
        hybrid/kmeans_1d_omp_mpi \
        data/dados_$name.csv data/dados_${name}_centroides_init.csv \
        50 1e-6 | grep "Tempo"
    
    echo "  2 processos MPI x 1 thread OpenMP:"
    OMP_NUM_THREADS=1 mpirun -np 2 --allow-run-as-root --oversubscribe \
        hybrid/kmeans_1d_omp_mpi \
        data/dados_$name.csv data/dados_${name}_centroides_init.csv \
        50 1e-6 | grep "Tempo"
    
    echo "  2 processos MPI x 2 threads OpenMP:"
    OMP_NUM_THREADS=2 mpirun -np 2 --allow-run-as-root --oversubscribe \
        hybrid/kmeans_1d_omp_mpi \
        data/dados_$name.csv data/dados_${name}_centroides_init.csv \
        50 1e-6 | grep "Tempo"
done

## Benchmark OpenMP + MPI

In [ ]:
%%bash
cd hybrid
mpicc -O2 -fopenmp kmeans_1d_omp_mpi.c -o kmeans_1d_omp_mpi -lm
echo "✓ OpenMP + MPI compilado"
ls -lh kmeans_1d_omp_mpi

## Compilar OpenMP + MPI

---

# 📊 PARTE 3: HYBRID (OpenMP + MPI) - CPU apenas

## Instalar OpenMPI

In [ ]:
%%bash
echo "========================================="
echo "BENCHMARK: HYBRID (OpenMP + MPI)"
echo "========================================="

datasets=("pequeno:10K:4" "medio:100K:8" "grande:1M:16")

for ds in "${datasets[@]}"; do
    IFS=':' read -r name N K <<< "$ds"
    echo ""
    echo "=== Dataset: ${name^^} (N=$N, K=$K) ==="
    
    # Teste com diferentes combinações MPI x OpenMP
    echo ""
    echo "  1 processo MPI x 2 threads OpenMP:"
    OMP_NUM_THREADS=2 mpirun -np 1 --allow-run-as-root --oversubscribe \
        hybrid/kmeans_1d_omp_mpi \
        data/dados_$name.csv data/dados_${name}_centroides_init.csv \
        50 1e-6 | grep "Tempo"
    
    echo "  2 processos MPI x 1 thread OpenMP:"
    OMP_NUM_THREADS=1 mpirun -np 2 --allow-run-as-root --oversubscribe \
        hybrid/kmeans_1d_omp_mpi \
        data/dados_$name.csv data/dados_${name}_centroides_init.csv \
        50 1e-6 | grep "Tempo"
    
    echo "  2 processos MPI x 2 threads OpenMP:"
    OMP_NUM_THREADS=2 mpirun -np 2 --allow-run-as-root --oversubscribe \
        hybrid/kmeans_1d_omp_mpi \
        data/dados_$name.csv data/dados_${name}_centroides_init.csv \
        50 1e-6 | grep "Tempo"
done

In [ ]:
import subprocess
import re
import csv
import os

print("========================================")
print("EXECUTANDO BENCHMARKS: OpenMP+MPI")
print("========================================")

# Criar diretório de resultados se não existir
os.makedirs('results', exist_ok=True)

# Armazenar resultados
resultados = []

datasets = [
    ("pequeno", "10K", "4"),
    ("medio", "100K", "8"),
    ("grande", "1M", "16")
]

configs = [
    ("2t1p", 1, 2),  # 1 processo, 2 threads
    ("1t2p", 2, 1),  # 2 processos, 1 thread
    ("2t2p", 2, 2),  # 2 processos, 2 threads
]

for name, N, K in datasets:
    print(f"\n=== Dataset: {name.upper()} (N={N}, K={K}) ===")
    
    for config_name, nprocs, nthreads in configs:
        print(f"  Config {config_name} ({nprocs}p x {nthreads}t): ", end="", flush=True)
        
        cmd = [
            "mpirun", "-np", str(nprocs), 
            "--allow-run-as-root", "--oversubscribe",
            "hybrid/kmeans_1d_omp_mpi",
            f"data/dados_{name}.csv",
            f"data/dados_{name}_centroides_init.csv",
            "50", "1e-6"
        ]
        
        env = os.environ.copy()
        env["OMP_NUM_THREADS"] = str(nthreads)
        
        try:
            result = subprocess.run(cmd, capture_output=True, text=True, env=env, timeout=60)
            
            # Extrair tempo do output
            match = re.search(r'Tempo total:\s*([\d.]+)\s*ms', result.stdout)
            if match:
                tempo = float(match.group(1))
                print(f"{tempo:.2f} ms")
                
                # Extrair SSE
                sse_match = re.search(r'SSE final:\s*([\d.]+)', result.stdout)
                sse = float(sse_match.group(1)) if sse_match else 0.0
                
                resultados.append({
                    'dataset': name,
                    'implementacao': 'OpenMP+MPI',
                    'config': config_name,
                    'tempo_ms': tempo,
                    'sse': sse
                })
            else:
                print("ERRO: não encontrou tempo no output")
                print(result.stdout)
        except Exception as e:
            print(f"ERRO: {e}")

# Salvar resultados em CSV
if resultados:
    csv_file = 'results/resultados_openmp_mpi.csv'
    with open(csv_file, 'w', newline='') as f:
        writer = csv.DictWriter(f, fieldnames=['dataset', 'implementacao', 'config', 'tempo_ms', 'sse'])
        writer.writeheader()
        writer.writerows(resultados)
    
    print(f"\n✓ Resultados salvos em: {csv_file}")
    print(f"✓ Total de {len(resultados)} medições coletadas")
else:
    print("\n⚠️  Nenhum resultado foi coletado!")

In [ ]:
!apt-get update -qq
!apt-get install -y openmpi-bin libopenmpi-dev -qq
print("✓ OpenMPI instalado")

## Compilar MPI + CUDA

In [ ]:
%%bash
cd hybrid
nvcc -O2 -arch=sm_75 -I/usr/lib/x86_64-linux-gnu/openmpi/include \
    kmeans_1d_mpi_cuda.cu -o kmeans_1d_mpi_cuda \
    -L/usr/lib/x86_64-linux-gnu/openmpi/lib -lmpi
echo "✓ MPI + CUDA compilado"
ls -lh kmeans_1d_mpi_cuda

## Benchmark MPI + CUDA

In [ ]:
%%bash
echo "========================================="
echo "BENCHMARK: HYBRID (MPI + CUDA)"
echo "========================================="

datasets=("pequeno:10K:4" "medio:100K:8" "grande:1M:16")

for ds in "${datasets[@]}"; do
    IFS=':' read -r name N K <<< "$ds"
    echo ""
    echo "=== Dataset: ${name^^} (N=$N, K=$K) ==="
    
    for PROCS in 1 2 4; do
        echo "  MPI $PROCS processos + CUDA:"
        mpirun -np $PROCS --allow-run-as-root hybrid/kmeans_1d_mpi_cuda \
            data/dados_$name.csv data/dados_${name}_centroides_init.csv \
            50 1e-6 256 | grep "Tempo"
    done
done

---

# 📊 COMPARAÇÃO FINAL

In [ ]:
import subprocess
import re
import pandas as pd

def extract_time(cmd):
    result = subprocess.run(cmd, shell=True, capture_output=True, text=True)
    match = re.search(r'Tempo.*?:\s*([\d.]+)\s*ms', result.stdout)
    return float(match.group(1)) if match else None

print("="*70)
print("RESUMO COMPARATIVO - Dataset GRANDE (1M pontos, K=16)")
print("="*70 + "\n")

results = []

# CUDA Puro
print("Testando CUDA...")
time_cuda = extract_time("cuda/kmeans_1d_cuda data/dados_grande.csv data/dados_grande_centroides_init.csv 50 1e-6 256")
results.append(('CUDA (256 TPB)', time_cuda))

# OpenMP + CUDA
for t in [1, 2, 4, 8]:
    print(f"Testando OpenMP({t}t) + CUDA...")
    time_hybrid = extract_time(
        f"OMP_NUM_THREADS={t} hybrid/kmeans_1d_omp_cuda "
        f"data/dados_grande.csv data/dados_grande_centroides_init.csv "
        f"50 1e-6 {t} 256"
    )
    results.append((f'OpenMP({t}t) + CUDA', time_hybrid))

# OpenMP + MPI (CPU apenas)
print("Testando OpenMP(2t) + MPI(1p)...")
time_hybrid = extract_time(
    "OMP_NUM_THREADS=2 mpirun -np 1 --allow-run-as-root --oversubscribe "
    "hybrid/kmeans_1d_omp_mpi "
    "data/dados_grande.csv data/dados_grande_centroides_init.csv 50 1e-6"
)
results.append(('OpenMP(2t) + MPI(1p)', time_hybrid))

print("Testando OpenMP(1t) + MPI(2p)...")
time_hybrid = extract_time(
    "OMP_NUM_THREADS=1 mpirun -np 2 --allow-run-as-root --oversubscribe "
    "hybrid/kmeans_1d_omp_mpi "
    "data/dados_grande.csv data/dados_grande_centroides_init.csv 50 1e-6"
)
results.append(('OpenMP(1t) + MPI(2p)', time_hybrid))

print("Testando OpenMP(2t) + MPI(2p)...")
time_hybrid = extract_time(
    "OMP_NUM_THREADS=2 mpirun -np 2 --allow-run-as-root --oversubscribe "
    "hybrid/kmeans_1d_omp_mpi "
    "data/dados_grande.csv data/dados_grande_centroides_init.csv 50 1e-6"
)
results.append(('OpenMP(2t) + MPI(2p)', time_hybrid))

# MPI + CUDA
for p in [1, 2, 4]:
    print(f"Testando MPI({p}p) + CUDA...")
    time_hybrid = extract_time(
        f"mpirun -np {p} --allow-run-as-root --oversubscribe "
        f"hybrid/kmeans_1d_mpi_cuda "
        f"data/dados_grande.csv data/dados_grande_centroides_init.csv 50 1e-6 256"
    )
    results.append((f'MPI({p}p) + CUDA', time_hybrid))

# Criar tabela
df = pd.DataFrame(results, columns=['Implementação', 'Tempo (ms)'])
df = df[df['Tempo (ms)'].notna()]

print("\n" + "="*70)
print(df.to_string(index=False))
print("="*70)

# Encontrar o mais rápido
if not df.empty:
    fastest = df.loc[df['Tempo (ms)'].idxmin()]
    print(f"\n🏆 Mais rápido: {fastest['Implementação']} com {fastest['Tempo (ms)']:.2f} ms")

---

## ✅ Benchmark Completo!

Todos os testes foram executados com:
- ✅ CUDA puro (GPU)
- ✅ OpenMP + CUDA (CPU multi-thread + GPU)
- ✅ MPI + CUDA (Distribuído + GPU)

**Próximos passos:**
1. Analisar os resultados
2. Comparar com resultados do Windows (Serial, OpenMP, MPI)
3. Gerar gráficos de speedup
4. Documentar no relatório

---

# 💾 SALVAR TODOS OS RESULTADOS

Execute esta célula para consolidar todos os resultados em um único arquivo CSV que o script de análise irá usar automaticamente.

In [ ]:
import subprocess
import re
import csv
import os
import pandas as pd

print("="*60)
print("EXECUTAR E CONSOLIDAR TODOS OS BENCHMARKS")
print("="*60)

# Criar diretório de resultados
os.makedirs('results', exist_ok=True)

all_results = []

datasets = [
    ("pequeno", "10K", "4"),
    ("medio", "100K", "8"),
    ("grande", "1M", "16")
]

# ============================================
# 1. CUDA PURO
# ============================================
print("\n[1/4] CUDA...")
for name, N, K in datasets:
    print(f"  {name.upper()}: ", end="", flush=True)
    cmd = f"cuda/kmeans_1d_cuda data/dados_{name}.csv data/dados_{name}_centroides_init.csv 50 1e-6 256"
    result = subprocess.run(cmd, shell=True, capture_output=True, text=True)
    
    match = re.search(r'Tempo(?:\s+total)?:\s*([\d.]+)\s*ms', result.stdout)
    sse_match = re.search(r'SSE final:\s*([\d.]+)', result.stdout)
    
    if match:
        tempo = float(match.group(1))
        sse = float(sse_match.group(1)) if sse_match else 0.0
        print(f"{tempo:.2f} ms")
        all_results.append({'dataset': name, 'implementacao': 'CUDA', 'config': '-', 'tempo_ms': tempo, 'sse': sse})
    else:
        print("ERRO")

# ============================================
# 2. OpenMP + CUDA
# ============================================
print("\n[2/4] OpenMP+CUDA...")
for name, N, K in datasets:
    print(f"  {name.upper()}")
    for threads in [1, 2, 4, 8]:
        print(f"    {threads}t: ", end="", flush=True)
        env = os.environ.copy()
        env['OMP_NUM_THREADS'] = str(threads)
        cmd = f"hybrid/kmeans_1d_omp_cuda data/dados_{name}.csv data/dados_{name}_centroides_init.csv 50 1e-6 256"
        result = subprocess.run(cmd, shell=True, capture_output=True, text=True, env=env)
        
        match = re.search(r'Tempo(?:\s+total)?:\s*([\d.]+)\s*ms', result.stdout)
        sse_match = re.search(r'SSE final:\s*([\d.]+)', result.stdout)
        
        if match:
            tempo = float(match.group(1))
            sse = float(sse_match.group(1)) if sse_match else 0.0
            print(f"{tempo:.2f} ms")
            all_results.append({'dataset': name, 'implementacao': 'OpenMP+CUDA', 'config': f'{threads}t', 'tempo_ms': tempo, 'sse': sse})
        else:
            print("ERRO")

# ============================================
# 3. MPI + CUDA
# ============================================
print("\n[3/4] MPI+CUDA...")
for name, N, K in datasets:
    print(f"  {name.upper()}: ", end="", flush=True)
    cmd = f"mpirun -np 1 --allow-run-as-root hybrid/kmeans_1d_mpi_cuda data/dados_{name}.csv data/dados_{name}_centroides_init.csv 50 1e-6 256"
    result = subprocess.run(cmd, shell=True, capture_output=True, text=True)
    
    match = re.search(r'Tempo(?:\s+total)?:\s*([\d.]+)\s*ms', result.stdout)
    sse_match = re.search(r'SSE final:\s*([\d.]+)', result.stdout)
    
    if match:
        tempo = float(match.group(1))
        sse = float(sse_match.group(1)) if sse_match else 0.0
        print(f"{tempo:.2f} ms")
        all_results.append({'dataset': name, 'implementacao': 'MPI+CUDA', 'config': '1p', 'tempo_ms': tempo, 'sse': sse})
    else:
        print("ERRO")

# ============================================
# 4. Carregar OpenMP+MPI (se existir)
# ============================================
print("\n[4/4] OpenMP+MPI...")
if os.path.exists('results/resultados_openmp_mpi.csv'):
    df_omp_mpi = pd.read_csv('results/resultados_openmp_mpi.csv')
    print(f"  ✓ Carregado: {len(df_omp_mpi)} medições")
    all_results.extend(df_omp_mpi.to_dict('records'))
else:
    print("  ⚠️  Arquivo não encontrado. Execute a célula de OpenMP+MPI primeiro!")

# ============================================
# SALVAR RESULTADOS
# ============================================
if all_results:
    df_final = pd.DataFrame(all_results)
    output_file = 'results/resultados_colab.csv'
    df_final.to_csv(output_file, index=False)
    
    print(f"\n{'='*60}")
    print(f"✅ RESULTADOS CONSOLIDADOS")
    print(f"{'='*60}")
    print(f"Arquivo: {output_file}")
    print(f"Total: {len(df_final)} medições")
    print(f"\nResumo por implementação:")
    print(df_final.groupby('implementacao').size())
    print(f"\n{'='*60}")
    print("✅ Agora execute: !python scripts/analise_completa.py")
    print(f"{'='*60}")
else:
    print("\n⚠️  NENHUM RESULTADO COLETADO!")
